In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 'pycocotools>=2.0.1'
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/torch_stable.html
1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# change to working tensorflow directory on the drive
%cd '/content/gdrive/My Drive/Tensorflow'

Mounted at /content/gdrive
/content/gdrive/My Drive/Tensorflow


In [ ]:
import pandas as pd

df = pd.read_csv('./workspace/training_demo/data/train_labels.csv')

df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,Dipto_ 111.jpg,1913,891,truck,7,327,350,804
1,Dipto_ 111.jpg,1913,891,truck,846,304,1067,671
2,Dipto_ 111.jpg,1913,891,truck,1316,396,1909,727
3,Dipto_ 111.jpg,1913,891,truck,697,455,852,641
4,Dipto_ 111.jpg,1913,891,three wheelers (CNG),326,502,689,847


In [ ]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2

# write a function that loads the dataset into detectron2's standard format
def get_microcontroller_dicts(csv_file, img_dir):
    df = pd.read_csv(csv_file)
    df['filename'] = df['filename'].map(lambda x: img_dir+x)

    classes = ['ambulance', 'auto-rickshaw', 'bicycle', 'bus', 'car', 'garbage van', 'human hauler', 'minibus', 'minivan', 'motorbike', 'Pickup', 'army vehicle', 'police car', 'rickshaw', 'scooter', 'Suv', 'taxi', 'three wheelers (CNG)', 'truck', 'van', 'wheelbarrow']

    df['class_int'] = df['class'].map(lambda x: classes.index(x))

    dataset_dicts = []
    for filename in df['filename'].unique().tolist():
        record = {}
        
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["height"] = height
        record["width"] = width

        objs = []
        for index, row in df[(df['filename']==filename)].iterrows():
          obj= {
              'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
              'bbox_mode': BoxMode.XYXY_ABS,
              'category_id': row['class_int'],
              "iscrowd": 0
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

classes = ['ambulance', 'auto-rickshaw', 'bicycle', 'bus', 'car', 'garbage van', 'human hauler', 'minibus', 'minivan', 'motorbike', 'Pickup', 'army vehicle', 'police car', 'rickshaw', 'scooter', 'Suv', 'taxi', 'three wheelers (CNG)', 'truck', 'van', 'wheelbarrow']

for d in ["train", "test"]:
  DatasetCatalog.register('/content/gdrive/My Drive/Tensorflow/workspace/training_demo/images/' + d, lambda d=d: get_microcontroller_dicts('/content/drive/My Drive/Tensorflow/workspace/training_demo/data/' + d + '_labels.csv', '/content/drive/My Drive/Tensorflow/workspace/training_demo/images/' + d+'/'))
  MetadataCatalog.get('/content/gdrive/My Drive/Tensorflow/workspace/training_demo/images/' + d).set(thing_classes=classes)
microcontroller_metadata = MetadataCatalog.get('/content/gdrive/My Drive/Tensorflow/workspace/training_demo/images/train')
print(microcontroller_metadata)
print(DatasetCatalog.register)

AssertionError: ignored

In [ ]:
import random
from detectron2.utils.visualizer import Visualizer

dataset_dicts = DatasetCatalog.get('/content/gdrive/My Drive/Tensorflow/workspace/training_demo/images/train')
print(dataset_dicts)
#for d in random.sample(dataset_dicts, 10):
#    img = cv2.imread(d["file_name"])
#    v = Visualizer(img[:, :, ::-1], metadata=microcontroller_metadata, scale=0.5)
#    v = v.draw_dataset_dict(d)
#    plt.figure(figsize = (14, 10))
#    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
#    plt.show()

OSError: ignored

In [ ]:
print()